In [58]:
import torch
import numpy as np
from math import sin, cos

SENTENCE = "I LIKE PIE VERY MUCH PIE ."
DIM_EMB = 512
torch.manual_seed(7)

vocab = dict()
i = 0
for token in SENTENCE.split():
    if token not in vocab:
        vocab[token] = i
        i += 1
vocab["<unk>"] = i
vocab["<s>"] = i + 1
print(vocab)

DICT_SIZE = len(vocab)

oneHotSent = np.zeros((len(SENTENCE.split()), DICT_SIZE), dtype=float)

for i in range(len(SENTENCE.split())):
    word = SENTENCE.split()[i]
    oneHotSent[i][vocab[word]] = 1

oneHotTorch = torch.as_tensor(oneHotSent, dtype=torch.float32)
    
#print(oneHotTorch)
#print(oneHotTorch.shape)
#print(oneHotTorch.dtype)

#DATA PREPARATION DONE

We_skeleton = torch.tensor(np.zeros((DICT_SIZE, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
We = torch.autograd.Variable(We_skeleton, requires_grad=True)
#print(We)
#print(We.shape)
#print(We.dtype)


Batch = oneHotTorch.mm(We)
Pos_emb = np.zeros(Batch.shape, dtype=float)
for p in range(len(SENTENCE.split())):
    for i in range(DIM_EMB//2):
        val = p/(10000**(2*i/DIM_EMB))
        Pos_emb[p][2*i] = sin(val)
        Pos_emb[p][2*i + 1] = cos(val)

Pos_emb_var = torch.autograd.Variable(torch.tensor(Pos_emb, dtype=torch.float32), requires_grad=False)
#print(Pos_emb_var)

#print(Batch)
#Scale batch up
Batch = Batch*(DIM_EMB**(1/2))
#print(Batch)
Batch = Batch + Pos_emb_var
#print(Batch)

#print(Batch.shape)
#print(Batch.dtype)

#Batch (in this case a one sentence batch) is prepared. Maxi calls this embedded sentence

#Create MASK HERE
#
mask = np.triu(np.ones((len(SENTENCE.split()), len(SENTENCE.split()))), k=1)
mask = torch.from_numpy(mask) == 0
print(mask)

# Create self attention
# also need to add a mask here
def add_transformer_head_with_mask(DIM_EMB, Batch, mask):
    Wq_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wq = torch.autograd.Variable(Wq_skeleton, requires_grad=True)

    Wk_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wk = torch.autograd.Variable(Wk_skeleton, requires_grad=True)

    Wv_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wv = torch.autograd.Variable(Wv_skeleton, requires_grad=True)

    Q = Batch.mm(Wq)
    V = Batch.mm(Wv)
    K = Batch.mm(Wk)
    
    tmp1 = Q.mm(torch.t(K))

    tmp = tmp1/(64**(1/2))
    #print(tmp)
    
    softM = torch.nn.Softmax(dim=-1)
    #apply maks here
    tmp = tmp.masked_fill(mask == 0, -1e9)
    
    #print(tmp)

    
    a = softM(tmp)
    print(a)

    z = a.mm(V)
    return z


print("EWADSDASDASDASDASDASDASDASDAS")
print(add_transformer_head_with_mask(DIM_EMB, Batch, mask))

{'I': 0, 'LIKE': 1, 'PIE': 2, 'VERY': 3, 'MUCH': 4, '.': 5, '<unk>': 6, '<s>': 7}
tensor([[1, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1]], dtype=torch.uint8)
EWADSDASDASDASDASDASDASDASDAS
tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0.]], grad_fn=<SoftmaxBackward>)
tensor([[ 1.3456e+03, -6.2462e+02,  2.6924e+02,  3.2292e+02,  5.9020e+02,
          2.8053e+02, -5.8125e+02, -7.0534e+02,  1.7737e+02, -3.4275e+02,
          2.5205e+02,  4.5682e+02, -2.4706e+02,  2.5111e+02, -1.0888e+03,
          5.9780e+02, -3.1436e+02,  4.6598e+02, -6.4631e+02,  6.1107e+01,
          2.8097e+02,  6.1181e+02, -5.4264e+02, -3.3123e+02, 